In [2]:
# Cell ▸ Download Aug-2018 → now (no scraping required)
import pathlib, requests, concurrent.futures as cf
from tqdm.auto import tqdm           # works with or without ipywidgets
from datetime import date, timedelta

ROOT       = pathlib.Path().resolve().parents[0]
PARCEL_DIR = ROOT / "data" / "raw" / "parcel-data"
PARCEL_DIR.mkdir(parents=True, exist_ok=True)

BASE = "https://apps.franklincountyauditor.com/Parcel_CSV/{y}/{m:02d}/Parcel.csv"
START = date(2018, 8, 1)                              # first NFIRS month
END   = date.today().replace(day=1)                   # latest complete month

# Build a list of (year, month) tuples
todo = []
cur = START
while cur <= END:
    todo.append((cur.year, cur.month))
    cur = (cur.replace(day=28) + timedelta(days=4)).replace(day=1)  # +1 month

from tqdm.auto import tqdm

def download_one(y, m):
    url  = f"{BASE.format(y=y, m=m)}"
    dest = PARCEL_DIR / f"parcel_{m:02d}_{y}.csv"
    if dest.exists():
        return f"skip {dest.name}"

    r = requests.get(url, stream=True, timeout=60)
    if r.status_code != 200:
        return f"⚠ {y}-{m:02d} missing"

    total = int(r.headers.get("content-length", 0))
    with open(dest, "wb") as f, tqdm(
            total=total, unit="B", unit_scale=True,
            desc=f"{dest.name}", leave=False) as bar:
        for chunk in r.iter_content(chunk_size=1 << 20):
            f.write(chunk)
            bar.update(len(chunk))
    return f"✓ {dest.name}"

with cf.ThreadPoolExecutor(max_workers=4) as pool:
    for msg in tqdm(pool.map(lambda ym: download_one(*ym), todo),  # ← unpack here
                    total=len(todo)):
        if msg.startswith("⚠"):
            print(msg)

  0%|          | 0/84 [00:00<?, ?it/s]

parcel_04_2019.csv:   0%|          | 0.00/232M [00:00<?, ?B/s]

parcel_07_2019.csv:   0%|          | 0.00/232M [00:00<?, ?B/s]

parcel_05_2019.csv:   0%|          | 0.00/232M [00:00<?, ?B/s]

parcel_06_2019.csv:   0%|          | 0.00/232M [00:00<?, ?B/s]

...

parcel_09_2019.csv:   0%|          | 0.00/232M [00:00<?, ?B/s]

parcel_10_2019.csv:   0%|          | 0.00/232M [00:00<?, ?B/s]

parcel_11_2019.csv:   0%|          | 0.00/233M [00:00<?, ?B/s]

parcel_12_2019.csv:   0%|          | 0.00/233M [00:00<?, ?B/s]

KeyboardInterrupt: 